In [1]:
import os, pandas as pd
from py2neo import Graph

In [2]:
files_path = "../Output/DataLLM"
files = os.listdir(files_path)
print(files)

['ODD01_keywords.csv', 'ODD02_keywords.csv', 'ODD03_keywords.csv', 'ODD04_keywords.csv', 'ODD05_keywords.csv']


In [3]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "@Martiale01"))

In [4]:
for data in files :
    df = pd.read_csv(f"{files_path}/{data}")    
    for _, row in df.iterrows():
        od = row['ODD']
        cible = row['Cible']
        indicateur = row['Indicateur']
        mot_cle = row['Mots-cles']

         # Nous utilisons MERGE au lieu de CREATE afin de créer les nœuds ODD, Cible, Indicateur, Mots-clés sans doublons. Si le nœud existe déjà, il ne sera pas recréé
        graph.run(f"MERGE (o:ODD {{titre: '{od}'}})")
        graph.run(f"MERGE (c:Cible {{titre: '{cible}'}})")
        graph.run(f"MERGE (i:Indicateur {{code: '{indicateur}'}})")
        graph.run(f'MERGE (m:MotCle {{mot: "{mot_cle}"}})')

        # Les relations entre les nœuds
        graph.run(f"""
        MATCH (o:ODD {{titre: "{od}"}}), (c:Cible {{titre: "{cible}"}}), 
              (i:Indicateur {{code: "{indicateur}"}}), (m:MotCle {{mot: "{mot_cle}"}})
        MERGE (o)-[:CONTIENT]->(c)
        MERGE (c)-[:CONTIENT]->(i)
        MERGE (i)-[:CONTIENT]->(m)
        """)